# constspeed_pipeline

**Short Introduction**  
The `constspeed_pipeline` function identifies and processes **constant-speed segments** from vehicle measurement data in order to **estimate drag parameters** (RLC parameters). It applies **optional pitch correction**, **time smoothing**, and **segment trimming**, then fits a **quadratic model** to the resulting data. This enables you to derive **aerodynamic drag** and **rolling resistance** coefficients, as well as additional losses under different suspension configurations.

---

## Parameters

- **data_cache** (`dict` or `None`)  
  A dictionary containing pre-loaded data keyed by filenames. If `None`, data is read directly from disk using `read_can_data`.

- **files** (`str` or `list` of `str`)  
  Can be either:  
  1. A **folder path** (string) to retrieve files with `get_can_files`, or  
  2. A **list of file paths** to be processed.  

- **speed_signal** (`str`)  
  Name of the speed column (e.g., `'vehicle_speed'` or `'vehicle_speed_gps'`) used to detect constant-speed segments.

- **speed_threshold** (`float`)  
  Numeric threshold defining how “constant” the speed must be to form a segment.

- **min_n_samples** (`int`)  
  Minimum number of consecutive data points required to constitute a valid constant-speed segment.

- **min_avg_speed** (`float`)  
  Minimum **average speed** (km/h) for a segment to be retained in the analysis.

- **do_pitch_correction** (`bool`, default=False)  
  If `True`, calculates a pitch-induced force (`mass*g*sin(pitch)`) and subtracts/adjusts it from the measured data.

- **cut_time** (`float` or `None`)  
  If not `None`, removes this many seconds from **both the start and the end** of each constant-speed segment (to avoid transient phases).

- **smoothing_kwargs** (`dict`, optional)  
  Arguments for **time-domain smoothing** on selected columns. If provided alongside `columns_to_smooth`, the specified columns will be filtered.  
  For example:  
  - `{'filter_type': 'moving_average', 'window_size': 3}`  
  - `{'filter_type': 'exponential_moving_average', 'alpha': 0.4}`  
  - `{'filter_type': 'savitzky_golay', 'window_size': 5, 'polyorder': 2}`  
  - `{'filter_type': 'lowpass', 'cutoff_frequency': 0.1, 'order': 2}`  

- **columns_to_smooth** (`list`, optional)  
  List of columns to smooth in time. If it includes `'vehicle_speed'`, that entry will be replaced internally with the actual `speed_signal`.

- **steering_angle_limit** (`float` or `None`)  
  If provided, excludes data points whose absolute steering angle (`steering_wheel_angle`) exceeds this limit.

- **select_suspension_level** (`int` or `None`)  
  If provided, discards segments that do not match this suspension level (e.g., `0` for very low, `1` for low, etc.).

- **vehicle_mass** (`float`, default=2300)  
  Vehicle mass (in kg). Used for pitch correction and drag-coefficient calculations.

- **frontal_area** (`float`, default=2.3)  
  Vehicle frontal area (in m²). Used to derive aerodynamic drag coefficients.

- **outlier_threshold** (`float` or `None`)  
  If provided, data points considered outliers in the **quadratic fitting** process will be removed based on this threshold.

- **loss_type** (`str`, default='rel')  
  Type of loss metric to compute in `calculate_all_losses`. Examples:
  - `'rel'` (relative)  
  - `'abs'` (absolute)  
  - `'sec_rmse'` (root mean squared error over the valid section)

- **generate_plots** (`bool`, default=False)  
  If `True`, creates figures showing segmentation results, ellipse plots of speed vs. accelerating force, and final braking-force fits.

- **verbose** (`bool`, default=False)  
  If `True`, logs additional step-by-step information.

- **filename_prefix** (`str`, default='constspeed')  
  A prefix for any saved figures (e.g., `'constspeed_segments_processed.pdf'`).

- **figsize_1**, **figsize_2** (`tuple`, default=(10, 5))  
  Figure sizes for the segment overview plots and the final braking-force fit plots, respectively.

---

## Return Values

A **17-element tuple** containing:

1. **`f0`**, **`f1`**, **`f2`**  
   Quadratic coastdown parameters derived from fitting a parabola of the form  
   \[
       F_\text{resist}(v) = f_0 + f_1 \cdot v + f_2 \cdot v^2
   \]

2. **`c_air`**, **`c_rr`**  
   Aerodynamic drag coefficient and rolling-resistance coefficient, computed from the fitted parameters.

3. **`metric_rm_steering_angle`**  
   Number of data points removed due to the steering angle limit.

4. **`metric_rm_suspension_level`**  
   Number of data points removed because their suspension level didn’t match `select_suspension_level`.

5. **`metric_rm_cut_time`**  
   Number of data points removed when trimming each segment by `cut_time` seconds at the start and end.

6. **`total_removed`**  
   Sum of all removed data points from steering angle, suspension level, and time-window filtering.

7. **`metric_outliers`**  
   Count of outlier points removed during the quadratic fitting process.

8. **`loss_very_low_suspension`, `loss_low_suspension`, `loss_medium_suspension`, `loss_high_suspension`**  
   Computed losses (relative or absolute) under different assumed suspension levels.

9. **`loss_CoC_values`, `loss_EPA_values`**  
   Additional computed losses relevant to other regulatory or comparative standards.

10. **`lowest_loss_name`**  
    The key (e.g., `'very_low_suspension'`) corresponding to the minimum loss.

11. **`elapsed_time`**  
    Total runtime (in seconds) for the entire pipeline.

In [ ]:
from modules.parametric_pipelines import constspeed_pipeline
from modules.data_handler import get_can_files

data_cache = None
files = get_can_files('data/folder_which_contains_recorded_driving_data_with_constant_speed_segments')  # Adjust to your path
speed_signal = 'vehicle_speed'
speed_threshold = 1
min_n_samples = 80
min_avg_speed = 0
do_pitch_correction = True
cut_time = 4
smoothing_kwargs = {'filter_type': 'moving_average', 'window_size': 80}
columns_to_smooth = [speed_signal]
steering_angle_limit = 10
select_suspension_level = None
vehicle_mass = 2380
frontal_area = 2.3
outlier_threshold = 1
loss_type = 'sec_rmse'
generate_plots = True
verbose = False
filename_prefix = 'constspeed'
figsize_1 = (5, 4)
figsize_2 = (10, 4)

results = constspeed_pipeline(
    data_cache=data_cache,
    files=files,
    speed_signal=speed_signal,
    speed_threshold=speed_threshold,
    min_n_samples=min_n_samples,
    min_avg_speed=min_avg_speed,
    do_pitch_correction=do_pitch_correction,
    cut_time=cut_time,
    smoothing_kwargs=smoothing_kwargs,
    columns_to_smooth=columns_to_smooth,
    steering_angle_limit=steering_angle_limit,
    select_suspension_level=select_suspension_level,
    vehicle_mass=vehicle_mass,
    frontal_area=frontal_area,
    outlier_threshold=outlier_threshold,
    loss_type=loss_type,
    generate_plots=generate_plots,
    verbose=verbose,
    filename_prefix=filename_prefix,
    figsize_1=figsize_1,
    figsize_2=figsize_2
)

# Unpack results
f0, f1, f2, c_air, c_rr, metric_rm_steering_angle, metric_rm_suspension_level, metric_rm_cut_time, total_removed, metric_outliers, loss_very_low_suspension, loss_low_suspension, loss_medium_suspension, loss_high_suspension, loss_CoC_values, loss_EPA_values, lowest_loss_name, elapsed_time = results

print(f'f0: {f0}, f1: {f1}, f2: {f2}')
print(f'c_air: {c_air}, c_rr: {c_rr}')
print('\n')
print(f'metric_rm_steering_angle: {metric_rm_steering_angle}')
print(f'metric_rm_suspension_level: {metric_rm_suspension_level}')
print(f'metric_rm_cut_time: {metric_rm_cut_time}')
print(f'total_removed: {total_removed}')
print(f'metric_outliers: {metric_outliers}')
print('\n')
print(f'loss_very_low_suspension: {loss_very_low_suspension}')
print(f'loss_low_suspension: {loss_low_suspension}')
print(f'loss_medium_suspension: {loss_medium_suspension}')
print(f'loss_high_suspension: {loss_high_suspension}')
print(f'loss_CoC_values: {loss_CoC_values}')
print(f'loss_EPA_values: {loss_EPA_values}')
print(f'lowest_loss_name: {lowest_loss_name}')
print('\n')
print(f'elapsed_time: {elapsed_time}')